In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from glob import glob

%matplotlib inline
sns.set()
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

# Importing Data

In [3]:
def getfiles(store, year):
    files = glob(f'../Water Daily Sales/Converted Data/{year} {store}/*')
    files.sort()
    data = pd.concat( (pd.read_excel(f, header=1, parse_dates=True) for f in files), ignore_index=True)
    return data

In [ ]:
feb2008 = pd.read_excel('../Water Daily Sales/Converted Data/2008 Lucky/Revenue_Lucky_DailySale Feb-08.xlsx', index_col='Product', parse_dates=True, header=1)

In [45]:
Lucky2008 = getfiles('Lucky', '2008')
Lucky2009 = getfiles('Lucky', '2009')
Lucky2010 = getfiles('Lucky', '2010')
Lucky2011 = getfiles('Lucky', '2011')
Lucky2012 = getfiles('Lucky', '2012')
Lucky2013 = getfiles('Lucky', '2013')
Lucky2014 = getfiles('Lucky', '2014')
Lucky2015 = getfiles('Lucky', '2015')
Lucky2016 = getfiles('Lucky', '2016')
Lucky2017 = getfiles('Lucky', '2017')
Lucky2018 = getfiles('Lucky', '2018')
Lucky2019 = getfiles('Lucky', '2019')
Lucky2020 = getfiles('Lucky', '2020')

In [ ]:
FM2010 = getfiles('FM', '2010')
FM2011 = getfiles('FM', '2011')
FM2012 = getfiles('FM', '2012')
FM2013 = getfiles('FM', '2013')
FM2014 = getfiles('FM', '2014')
FM2015 = getfiles('FM', '2015')
FM2016 = getfiles('FM', '2016')
FM2017 = getfiles('FM', '2017')
FM2018 = getfiles('FM', '2018')
FM2019 = getfiles('FM', '2019')
FM2020 = getfiles('FM', '2020')

In [ ]:
Lion2016 = getfiles('Lion', '2016')
Lion2017 = getfiles('Lion', '2017')
Lion2018 = getfiles('Lion', '2018')
Lion2019 = getfiles('Lion', '2019')
Lion2020 = getfiles('Lion', '2020')

# Cleaning Up Data

## Old Model Dataset

In [ ]:
Lucky2008 = Lucky2008.rename(columns={'Product': 'Date'})

In [ ]:
Lucky2008 = Lucky2008.set_index('Date')

In [ ]:
Lucky2008.index = pd.to_datetime(Lucky2008.index)

In [ ]:
Lucky2008 = Lucky2008.sort_values(by='Date')

In [ ]:
Lucky2008.index.frq = 'D'

In [ ]:
#Lucky2008.columns

In [ ]:
Lucky2008 = Lucky2008.drop([1], axis=1)

In [ ]:
missing = Lucky2008.isnull().sum()/len(Lucky2008)*100
missing = missing.drop(missing[missing == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio': missing})

In [ ]:
Lucky2008 = Lucky2008.drop(columns=Lucky2008.filter(like='Unnamed').columns)

In [ ]:
Crocks = [c for c in Lucky2008.columns if 'Crocks' in c]
Stands = [k for k in Lucky2008.columns if 'Stands' in k]
Crocks_Stands = Crocks+Stands
Crocks_Stands

In [ ]:
Lucky2008[Crocks_Stands] = Lucky2008[Crocks_Stands].fillna(0)
Lucky2008['Crocks_Stands'] = 0
for crock in Crocks_Stands:
    Lucky2008['Crocks_Stands'] += Lucky2008[crock]
Lucky2008['Crocks_Stands']

In [ ]:
Lucky2008 = Lucky2008.drop(Crocks_Stands, axis=1)

In [ ]:
caps = [k for k in Lucky2008.columns if 'Cap' in k]
caps

In [ ]:
Lucky2008[caps] = Lucky2008[caps].fillna(0)
Lucky2008['Cap'] = 0
for cap in caps:
    Lucky2008['Cap'] += Lucky2008[cap]
Lucky2008['Cap']

In [ ]:
Lucky2008 = Lucky2008.drop(caps, axis=1)

In [ ]:
Lucky2008 = Lucky2008.fillna(0)

In [ ]:
Lucky2008['Weekday'] = pd.to_datetime(Lucky2008['Weekday'].loc[])

# Lucky 2020 (Standard)

In [29]:
Lucky2020 = getfiles('Lucky', '2020')

In [30]:
Lucky2020 = Lucky2020.set_index('Date')
Lucky2020.index = pd.to_datetime(Lucky2020.index)
Lucky2020 = Lucky2020.sort_values(by='Date')
Lucky2020.index.frq = 'D'

In [31]:
Lucky2020 = Lucky2020.drop(columns=Lucky2020.filter(like='Unnamed').columns)

In [159]:
missing = Lucky2020.isnull().sum()/len(Lucky2020)*100
missing = missing.drop(missing[missing==0].index).sort_values(ascending=False)
missing_values = pd.DataFrame({'Missing Ratio': missing})

In [32]:
cols = [1, 'Total PPE Sales','Cash Receipt (record)','Cash Over (Short)','Cash Paid Out','Gross Receipts (no sales tax)','Credit card Receipt (record)','  Total Water sales','Weekday','Date.1' ,'Number of customers', 'ASP Machine', 'ASP Water Revenue', 'Total Revenue', 'Prepaid water percentage', 'ASP Non Water Revenue', 'Sales Tax', 'Revenue per hour', 'Net Sales']
Lucky2020 = Lucky2020.drop(cols, axis=1)

In [33]:
Lucky2020 = Lucky2020.fillna(0)

## Combining Columns

In [23]:
#alkaline water

Lucky2020['Alkaline Water Sales'] = Lucky2020['0-5-gal-alkaline'] + Lucky2020['1-gal-alkaline']

In [24]:
Lucky2020 = Lucky2020.drop(['0-5-gal-alkaline', '1-gal-alkaline'], axis=1)

In [25]:
#normal water (cash)

Lucky2020['Water Sales'] = Lucky2020['0-5-gal-water'] + Lucky2020['5-gal-water'] + Lucky2020['1-gal-water']

In [26]:
Lucky2020 = Lucky2020.drop(['0-5-gal-water', '1-gal-water', '5-gal-water'], axis=1)

In [27]:
rename_cols = {'prepaid-100-alkaline-water':'Prepaid Alkaline',
               'Prepaid Water':'Prepaid Water Gallons', 'Total Water Gallon sale':'Total Water Gallons',
               'Alkaline':'Alkaline Gallons', 'Water Cash Gallon':'Water Cash Gallons',
              ' Total non water sales': 'Non-Water Sales'}
Lucky2020 = Lucky2020.rename(columns=rename_cols)

In [34]:
#merging crocks and stands
crocks_stands = []
for col in Lucky2020.columns:
    if ('crock' in col.lower()) or ('stand' in col.lower()):
        crocks_stands.append(col)

In [168]:
Lucky2020['stand-tall'] = Lucky2020['stand-tall'].replace('                                                                                        ', 0)
Lucky2020['stand-tall'] = Lucky2020['stand-tall'].astype('float64')

In [169]:
Lucky2020['Crocks_Stands'] = 0
for i in crocks_stands:
    Lucky2020['Crocks_Stands'] += Lucky2020[i] 

In [170]:
Lucky2020 = Lucky2020.drop(crocks_stands, axis=1)

In [171]:
#merging caps
all_caps = [k for k in Lucky2020.columns if 'cap' in k.lower()]

In [172]:
Lucky2020['Caps'] = 0
for cap in all_caps:
    Lucky2020['Caps'] += Lucky2020[cap]

In [173]:
Lucky2020 = Lucky2020.drop(all_caps, axis=1)

In [174]:
Lucky2020.columns

Index(['Shift', 'Prepaid Alkaline', 'Card-150-gallons', 'bot1-2',
       'Bottle-bpa-1-gal', 'Bottle-BPA-2-gal', 'Bottle BPA-2-gal-refrigerator',
       'Bottle-BPA-3-gal-faucet', 'Bottle-BPA-3-gal-refrigerator',
       'Bottle-BPA-3-gal-round-faucet', 'Bottle-BPA-3-gal-round-short',
       'Bottle BPA-3 Gal Tall', 'Bottle-BPA-5 Round',
       'Bottle-BPA-5-gal-with-faucet', 'Bottle BPA-5-gal-Big-Faucet', 'faucet',
       'atm-fee', 'Non-Water Sales', 'Mask 1', 'Employee', 'Alkaline Gallons',
       'Prepaid Water Gallons', 'Water Cash Gallons', 'Total Water Gallons',
       'prepaid-150-gal', 'Bottle-BPA half gallon',
       'Bottle-BPA-2-gal-refrigerator', 'Bottle-BPA-3-Gal-Tall',
       'Bottle-BPA-5-gal', 'Bottle-BPA-5-gal-Big-Faucet', 'Faucet',
       'Sanitizer 4oz', 'Custom Item', 'Alkaline Water Sales', 'Water Sales',
       'Crocks_Stands', 'Caps'],
      dtype='object')

In [175]:
#combining bottle names

Lucky2020['1/2 Gal Bottle'] = Lucky2020['bot1-2'] + Lucky2020['Bottle-BPA half gallon']
Lucky2020['2 Gal Bottle Refrig'] = Lucky2020['Bottle BPA-2-gal-refrigerator'] + Lucky2020['Bottle-BPA-2-gal-refrigerator']
Lucky2020['3 Gal Bottle Tall'] = Lucky2020['Bottle BPA-3 Gal Tall'] + Lucky2020['Bottle-BPA-3-Gal-Tall']
Lucky2020['5 Gal Bottle Big'] = Lucky2020['Bottle BPA-5-gal-Big-Faucet'] + Lucky2020['Bottle-BPA-5-gal-Big-Faucet']
Lucky2020['5 Gal Bottle'] = Lucky2020['Bottle-BPA-5 Round'] + Lucky2020['Bottle-BPA-5-gal']
Lucky2020['Faucet'] += Lucky2020['faucet']
Lucky2020['3 Gal Bottle Tall-Faucet'] = Lucky2020['Bottle-BPA-3-gal-round-faucet'] + Lucky2020['Bottle-BPA-3-gal-faucet']
Lucky2020['Prepaid Water'] = Lucky2020['Card-150-gallons'] + Lucky2020['prepaid-150-gal']

rename_cols = {'atm-fee':'ATM Fee',
        'Bottle-BPA-3-gal-round-short':'3 Gal Bottle Round-Short',
        'Bottle-BPA-3-gal-refrigerator':'3 Gal Bottle Refrig',
        'Bottle-BPA-5-gal-with-faucet':'5 Gal Bottle-Faucet',
        'Bottle-bpa-1-gal':'1 Gal Bottle',
        'Bottle-BPA-2-gal':'2 Gal Bottle',
        'Sanitizer 4oz':'Sanitizer', 'Mask 1': 'Mask', 'Non-Water Sales':'Non-Water Revenue'}
Lucky2020 = Lucky2020.rename(columns=rename_cols)

In [176]:
drop_cols = ['bot1-2', 'Bottle-BPA half gallon', 'Bottle BPA-2-gal-refrigerator',
            'Bottle-BPA-2-gal-refrigerator', 'Bottle BPA-3 Gal Tall', 'Bottle-BPA-3-Gal-Tall',
            'Bottle BPA-5-gal-Big-Faucet', 'Bottle-BPA-5-gal-Big-Faucet', 'faucet',
            'Bottle-BPA-5 Round', 'Bottle-BPA-5-gal', 'Card-150-gallons', 'prepaid-150-gal',
            'Bottle-BPA-3-gal-faucet', 'Bottle-BPA-3-gal-round-faucet']
Lucky2020 = Lucky2020.drop(columns=drop_cols, axis=1)

In [177]:
Lucky2020['Employee'] = Lucky2020['Employee'].replace(0, 'Bao Cam')

In [178]:
Lucky2020['Weekday'] = Lucky2020.index.day_name()
Lucky2020['Weekday'] = Lucky2020['Weekday'].apply(lambda x: x[:3])

In [180]:
water_rev = ['Water Sales', 'Prepaid Water', 'Alkaline Water Sales', 'Prepaid Alkaline']
Lucky2020['Total Water Revenue'] = 0
for sales in water_rev:
    Lucky2020['Total Water Revenue'] += Lucky2020[sales]

Lucky2020['Total Revenue'] = Lucky2020['Total Water Revenue'] + Lucky2020['Non-Water Revenue']

In [181]:
og_cols = Lucky2020.columns.tolist()
og_cols

['Shift',
 'Prepaid Alkaline',
 '1 Gal Bottle',
 '2 Gal Bottle',
 '3 Gal Bottle Refrig',
 '3 Gal Bottle Round-Short',
 '5 Gal Bottle-Faucet',
 'ATM Fee',
 'Non-Water Revenue',
 'Mask',
 'Employee',
 'Alkaline Gallons',
 'Prepaid Water Gallons',
 'Water Cash Gallons',
 'Total Water Gallons',
 'Faucet',
 'Sanitizer',
 'Custom Item',
 'Alkaline Water Sales',
 'Water Sales',
 'Crocks_Stands',
 'Caps',
 '1/2 Gal Bottle',
 '2 Gal Bottle Refrig',
 '3 Gal Bottle Tall',
 '5 Gal Bottle Big',
 '5 Gal Bottle',
 '3 Gal Bottle Tall-Faucet',
 'Prepaid Water',
 'Weekday',
 'Total Water Revenue',
 'Total Revenue']

In [188]:
reordered = ['Weekday', 'Shift', 'Employee',
             'Water Sales', 'Prepaid Water', 'Alkaline Water Sales', 'Prepaid Alkaline',
             '1/2 Gal Bottle','1 Gal Bottle', '2 Gal Bottle', '2 Gal Bottle Refrig',
             '3 Gal Bottle Refrig', '3 Gal Bottle Round-Short', '3 Gal Bottle Tall', '3 Gal Bottle Tall-Faucet',
             '5 Gal Bottle', '5 Gal Bottle-Faucet', '5 Gal Bottle Big',
             'Crocks_Stands', 'Caps', 'Faucet', 'Mask', 'Sanitizer', 'Custom Item', 'ATM Fee',
             'Total Water Revenue', 'Non-Water Revenue', 'Total Revenue',
             'Water Cash Gallons', 'Prepaid Water Gallons', 'Alkaline Gallons', 'Total Water Gallons']

In [190]:
def uncommon_items(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    if(set1 ^ set2):
        return('Uncommon elements: ' + str(set1 ^ set2))
    else:
        return('No uncommon elements')
uncommon_items(og_cols, reordered)

'No uncommon elements'

In [191]:
Lucky2020 = Lucky2020[reordered]

In [192]:
Lucky2020.head()

,Weekday,Shift,Employee,Water Sales,Prepaid Water,Alkaline Water Sales,Prepaid Alkaline,1/2 Gal Bottle,1 Gal Bottle,2 Gal Bottle,2 Gal Bottle Refrig,3 Gal Bottle Refrig,3 Gal Bottle Round-Short,3 Gal Bottle Tall,3 Gal Bottle Tall-Faucet,5 Gal Bottle,5 Gal Bottle-Faucet,5 Gal Bottle Big,Crocks_Stands,Caps,Faucet,Mask,Sanitizer,Custom Item,ATM Fee,Total Water Revenue,Non-Water Revenue,Total Revenue,Water Cash Gallons,Prepaid Water Gallons,Alkaline Gallons,Total Water Gallons
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01,Wed,Morning,Bao Cam,46.20,78.0,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,17.95,0.00,0.0,0.0,1.38,0.0,0.0,0.0,0.0,0.00,124.20,19.33,143.53,44.0,0.0,0,44.0
2020-01-01,Wed,Evening,Bao Cam,29.05,117.0,6.0,0,5.95,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,21.95,0.0,0.0,0.69,0.0,0.0,0.0,0.0,0.00,152.05,28.59,180.64,27.0,0.0,6,33.0
2020-01-02,Thu,Morning,Bao Cam,85.23,195.0,2.0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,282.23,0.00,282.23,100.0,0.0,2,102.0
2020-01-02,Thu,Evening,Bao Cam,80.50,117.0,5.0,0,5.95,7.5,0.0,0.0,0.0,0.0,0.0,17.95,35.90,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.75,202.50,68.05,270.55,86.0,0.0,5,91.0
2020-01-03,Fri,Morning,Bao Cam,102.73,156.0,3.0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,43.90,0.0,0.0,0.69,0.0,0.0,0.0,0.0,0.75,261.73,45.34,307.07,246.0,0.0,3,249.0


# Creating Function to Standardize Data

In [36]:
Lucky2020 = getfiles('Lucky', '2020')

In [42]:
reordered = ['Weekday', 'Shift', 'Employee',
         'Water Sales', 'Prepaid Water', 'Alkaline Water Sales', 'Prepaid Alkaline',
         '1/2 Gal Bottle','1 Gal Bottle', '2 Gal Bottle', '2 Gal Bottle Refrig',
         '3 Gal Bottle Refrig', '3 Gal Bottle Round-Short', '3 Gal Bottle Tall', '3 Gal Bottle Tall-Faucet',
         '5 Gal Bottle', '5 Gal Bottle-Faucet', '5 Gal Bottle Big',
         'Crocks_Stands', 'Caps', 'Faucet', 'Mask', 'Sanitizer', 'Custom Item', 'ATM Fee',
         'Total Water Revenue', 'Non-Water Revenue', 'Total Revenue',
         'Water Cash Gallons', 'Prepaid Water Gallons', 'Alkaline Gallons', 'Total Water Gallons']
def standardize_data(dataset):
    dataset = dataset.set_index('Date')
    dataset.index = pd.to_datetime(dataset.index)
    dataset = dataset.sort_values(by='Date')
    dataset.index.frq = 'D'

    dataset = dataset.drop(columns=dataset.filter(like='Unnamed').columns)
    dataset = dataset.fillna(0)
    cols = [1, 'Total PPE Sales','Cash Receipt (record)','Cash Over (Short)','Cash Paid Out','Gross Receipts (no sales tax)','Credit card Receipt (record)','  Total Water sales','Weekday','Date.1' ,'Number of customers', 'ASP Machine', 'ASP Water Revenue', 'Total Revenue', 'Prepaid water percentage', 'ASP Non Water Revenue', 'Sales Tax', 'Revenue per hour', 'Net Sales']
    dataset = dataset.drop(cols, axis=1)

    rename_cols = {'prepaid-100-alkaline-water':'Prepaid Alkaline',
               'Prepaid Water':'Prepaid Water Gallons', 'Total Water Gallon sale':'Total Water Gallons',
               'Alkaline':'Alkaline Gallons', 'Water Cash Gallon':'Water Cash Gallons',
              ' Total non water sales': 'Non-Water Sales'}
    dataset = dataset.rename(columns=rename_cols)
    
    crocks_stands = []
    for col in dataset.columns:
        if ('crock' in col.lower()) or ('stand' in col.lower()):
            crocks_stands.append(col)
    dataset['stand-tall'] = dataset['stand-tall'].replace('                                                                                        ', 0)
    dataset['stand-tall'] = dataset['stand-tall'].astype('float64')
    dataset['Crocks_Stands'] = 0
    for i in crocks_stands:
        dataset['Crocks_Stands'] += dataset[i]
    dataset = dataset.drop(crocks_stands, axis=1)

    all_caps = [k for k in dataset.columns if 'cap' in k.lower()]
    dataset['Caps'] = 0
    for cap in all_caps:
        dataset['Caps'] += dataset[cap]
    dataset = dataset.drop(all_caps, axis=1)

    dataset['Alkaline Water Sales'] = dataset['0-5-gal-alkaline'] + dataset['1-gal-alkaline']
    dataset['Water Sales'] = dataset['0-5-gal-water'] + dataset['5-gal-water'] + dataset['1-gal-water']
    dataset['1/2 Gal Bottle'] = dataset['bot1-2'] + dataset['Bottle-BPA half gallon']
    dataset['2 Gal Bottle Refrig'] = dataset['Bottle BPA-2-gal-refrigerator'] + dataset['Bottle-BPA-2-gal-refrigerator']
    dataset['3 Gal Bottle Tall'] = dataset['Bottle BPA-3 Gal Tall'] + dataset['Bottle-BPA-3-Gal-Tall']
    dataset['5 Gal Bottle Big'] = dataset['Bottle BPA-5-gal-Big-Faucet'] + dataset['Bottle-BPA-5-gal-Big-Faucet']
    dataset['5 Gal Bottle'] = dataset['Bottle-BPA-5 Round'] + dataset['Bottle-BPA-5-gal']
    dataset['Faucet'] += dataset['faucet']
    dataset['3 Gal Bottle Tall-Faucet'] = dataset['Bottle-BPA-3-gal-round-faucet'] + dataset['Bottle-BPA-3-gal-faucet']
    dataset['Prepaid Water'] = dataset['Card-150-gallons'] + dataset['prepaid-150-gal']

    rename_cols = {'atm-fee':'ATM Fee',
        'Bottle-BPA-3-gal-round-short':'3 Gal Bottle Round-Short',
        'Bottle-BPA-3-gal-refrigerator':'3 Gal Bottle Refrig',
        'Bottle-BPA-5-gal-with-faucet':'5 Gal Bottle-Faucet',
        'Bottle-bpa-1-gal':'1 Gal Bottle',
        'Bottle-BPA-2-gal':'2 Gal Bottle',
        'Sanitizer 4oz':'Sanitizer', 'Mask 1': 'Mask', 'Non-Water Sales':'Non-Water Revenue'}
    dataset = dataset.rename(columns=rename_cols)

    drop_cols = ['bot1-2', 'Bottle-BPA half gallon', 'Bottle BPA-2-gal-refrigerator',
            'Bottle-BPA-2-gal-refrigerator', 'Bottle BPA-3 Gal Tall', 'Bottle-BPA-3-Gal-Tall',
            'Bottle BPA-5-gal-Big-Faucet', 'Bottle-BPA-5-gal-Big-Faucet', 'faucet',
            'Bottle-BPA-5 Round', 'Bottle-BPA-5-gal', 'Card-150-gallons', 'prepaid-150-gal',
            'Bottle-BPA-3-gal-faucet', 'Bottle-BPA-3-gal-round-faucet']
    dataset = dataset.drop(columns=drop_cols, axis=1)


    water_rev = ['Water Sales', 'Prepaid Water', 'Alkaline Water Sales', 'Prepaid Alkaline']
    dataset['Total Water Revenue'] = 0
    for sales in water_rev:
        dataset['Total Water Revenue'] += dataset[sales]
    dataset['Total Revenue'] = dataset['Total Water Revenue'] + dataset['Non-Water Revenue']

    dataset['Employee'] = dataset['Employee'].replace(0, 'Bao Cam')
    dataset['Weekday'] = dataset.index.day_name()
    dataset['Weekday'] = dataset['Weekday'].apply(lambda x: x[:3])
    dataset = dataset[reordered]
    return dataset

In [43]:
Lucky2020 = standardize_data(Lucky2020)

In [44]:
Lucky2020.head()

,Weekday,Shift,Employee,Water Sales,Prepaid Water,Alkaline Water Sales,Prepaid Alkaline,1/2 Gal Bottle,1 Gal Bottle,2 Gal Bottle,2 Gal Bottle Refrig,3 Gal Bottle Refrig,3 Gal Bottle Round-Short,3 Gal Bottle Tall,3 Gal Bottle Tall-Faucet,5 Gal Bottle,5 Gal Bottle-Faucet,5 Gal Bottle Big,Crocks_Stands,Caps,Faucet,Mask,Sanitizer,Custom Item,ATM Fee,Total Water Revenue,Non-Water Revenue,Total Revenue,Water Cash Gallons,Prepaid Water Gallons,Alkaline Gallons,Total Water Gallons
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01,Wed,Morning,Bao Cam,46.20,78.0,0.0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,17.95,0.00,0.0,0.0,1.38,0.0,0.0,0.0,0.0,0.00,124.20,19.33,143.53,44.0,0.0,0,44.0
2020-01-01,Wed,Evening,Bao Cam,29.05,117.0,6.0,0,5.95,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,21.95,0.0,0.0,0.69,0.0,0.0,0.0,0.0,0.00,152.05,28.59,180.64,27.0,0.0,6,33.0
2020-01-02,Thu,Morning,Bao Cam,85.23,195.0,2.0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,282.23,0.00,282.23,100.0,0.0,2,102.0
2020-01-02,Thu,Evening,Bao Cam,80.50,117.0,5.0,0,5.95,7.5,0.0,0.0,0.0,0.0,0.0,17.95,35.90,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.75,202.50,68.05,270.55,86.0,0.0,5,91.0
2020-01-03,Fri,Morning,Bao Cam,102.73,156.0,3.0,0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,43.90,0.0,0.0,0.69,0.0,0.0,0.0,0.0,0.75,261.73,45.34,307.07,246.0,0.0,3,249.0
